In [ ]:
import sys
import os
sys.path.append('/home/onyxia/work/libsigma')
import geopandas as gpd
from shapely.geometry import box
from classification import rasterization, get_samples_from_roi
import read_and_write as rw
from osgeo import gdal

# --- Paramètres ---
input_vector_path = "../../data/project/FORMATION_VEGETALE.shp"  # Chemin vers la BD Forêt
ref_raster_path = "../../data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B2.tif"  # Raster Sentinel-2 de référence
output_rasterized_path = "../results/data/sample/rasterized_roi.tif"
output_folder = "../results/data/sample"
output_file = os.path.join(output_folder, "Sample_BD_foret_T31TCJ.shp")

# Liste des classes valides (colonne "Code TFV BD forêt")
codes_tfv_valides = [
    "Forêt fermée d’un autre feuillu pur",
    "Forêt fermée de châtaignier pur",
    "Forêt fermée de hêtre pur",
    "Forêt fermée de chênes décidus purs",
    "Forêt fermée de robinier pur",
    "Peuplerai",
    "Forêt fermée à mélange de feuillus",
    "Forêt fermée de feuillus purs en îlots",
    "Forêt fermée d’un autre conifère pur autre que pin",
    "Forêt fermée de mélèze pur",
    "Forêt fermée de sapin ou épicéa",
    "Forêt fermée à mélange d’autres conifères",
    "Forêt fermée d’un autre pin pur",
    "Forêt fermée de pin sylvestre pur",
    "Forêt fermée à mélange de pins purs",
    "Forêt fermée de douglas pur",
    "Forêt fermée de pin laricio ou pin noir pur",
    "Forêt fermée de pin maritime pur",
    "Forêt fermée à mélange de conifères",
    "Forêt fermée de conifères en îlots",
    "Forêt fermée à mélange de conifères prépondérants et feuillus",
    "Forêt fermée à mélange de feuillus prépondérants et conifères"
]

# Création du dossier de sortie si inexistant
os.makedirs(output_folder, exist_ok=True)

# --- Étape 1 : Filtrer les polygones par les classes TFV ---
print("Étape 1: Filtrage des classes TFV valides...")
bd_foret = gpd.read_file(input_vector_path)
bd_foret_filtered = bd_foret[bd_foret["TFV"].isin(codes_tfv_valides)]

# --- Étape 2 : Rasterisation pour générer un masque ---
print("Étape 2: Rasterisation des polygones...")
rasterization(
    in_vector=input_vector_path,
    ref_image=ref_raster_path,
    out_image=output_rasterized_path,
    field_name="TFV"
)

# --- Étape 3 : Extraction des échantillons avec get_samples_from_roi ---
print("Étape 3: Extraction des pixels avec get_samples_from_roi...")
# Utilisation de get_samples_from_roi pour extraire les pixels correspondant aux polygones
X, Y, t = get_samples_from_roi(
    raster_name=ref_raster_path,
    roi_name=output_rasterized_path,
    value_to_extract=None,  # Extraire toutes les valeurs correspondant aux polygones
    bands=None,             # Extraire toutes les bandes
    output_fmt='full_matrix'
)

print(f"Nombre de pixels extraits : {X.shape[0]}")

# --- Étape 4 : Créer un fichier vecteur final avec les attributs demandés ---
print("Étape 4: Création du fichier vecteur final...")
# Convertir les résultats en GeoDataFrame pour sauvegarde
geometry = [box(t[1][i], t[0][i], t[1][i], t[0][i]) for i in range(len(t[0]))]
sample_gdf = gpd.GeoDataFrame({"Code": Y.flatten(), "Nom": "Classified Pixel"}, geometry=geometry)

# Sauvegarder le fichier vecteur
sample_gdf.to_file(output_file, driver="ESRI Shapefile")

print(f"Fichier final créé : {output_file}")


Étape 1: Filtrage des classes TFV valides...
Étape 2: Rasterisation des polygones...
Étape 3: Extraction des pixels avec get_samples_from_roi...
Impossible to open ../results/data/sample/rasterized_roi.tif


/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'RasterXSize'